# Spark Basics 1

This notebook inroduces two fundamental objects in Spark:
* The Spark Context

* The Resilient Distributed DataSet or RDD

### Spark Context
* Spark is complex distributed doftware. 
* The python interface to spark is called **pyspark**
* **SparkContext** is a python class, defined as part of **pyspark** which manages the communication between the user's program and spark.

We start by creating a **SparkContext** object named **sc**. In this case we create a spark context that uses 3 *executors*

In [1]:
from pyspark import SparkContext
sc = SparkContext(master="local[3]")
sc

### Only one sparkContext at a time!
When you run spark in local mode, you can have only a single context at a time. Therefor, if you want to use spark in a second notebook, you should first stop the one you are using here. This is what the method `.stop()` is for.

In [2]:
# sc.stop() #commented out so that you don't stop your context by mistake

<h3>RDDs</h3>

<p>RDD (or Resilient Distributed DataSet) is the main novel data structure in Spark. You can think of it as a list whose elements are stored on several computers.</p>

<p><img alt="" src="Figures/SparkContextAndRDD.jpg" style="height:324px; width:900px" /></p>


The elements of each `RDD` are distributed across the **worker nodes** which are the nodes that perform the actual computations. This notebook, however, is running on the **Driver node**. As the RDD is not stored on the driver-node you cannot access it directly. The variable name `RDD` is really just a pointer to a python object which holds the information regardnig the actual location of the elements.

#### Parallelize 
* Simplest way to create an RDD.
* The method `A=sc.parallelize(L)`, creates an RDD named `A` from list `L`.
* `A` is an RDD of type `ParallelCollectionRDD`.

In [3]:
A=sc.parallelize(range(3))
A

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:423

#### Collect

* RDD content is distributed among all executors.
* `collect()` is the inverse of `parallelize()'
* collects the elements of the RDD
* Returns a list


In [4]:
L=A.collect()
print type(L)
print L

<type 'list'>
[0, 1, 2]


### Using `.collect()` eliminates the benefits of parallelism
It is often tempting to `.collect()` and RDD, make it into a list, and then process the list using standard python. However, note that this means that you are using only the head node to perform the computation which means that you are not getting any benefit from spark.

Using RDD operations, as described below, **will** make use of all of the computers at your disposal.

### Map
* applies a given operation to each element of an RDD
* parameter is the function defining the operation.
* returns a new RDD.
* Operation performed in parallel on all executors.
* Each executor operates on the data **local** to it.

**Note:** Here we are using **lambda** functions, later we will see that regular functions can also be used.

For more on lambda function see [here](http://www.secnetix.de/olli/Python/lambda_functions.hawk)

In [5]:
A.map(lambda x: x*x).collect()

[0, 1, 4]

#### Excercise 1

1. Write a function called `mapcos` that has a single paramater: an RDD of numbers, computes the `cos()` (cosine) and returns it as a list.

```cos(A)``` should produce the output
    
```
    [1.0, 0.54030..., -0.41614...]
```

**Approximate answers** The reason that the numbers above end with `...` is that, when computing with floating point numbers, there are always roundoff errors, and the roundoff errors may depend on the architecture of the computer you are using. To avoid this problem, we test the answers you produce within a tolerance of 0.1%, in other words if your answer is $a$ and the correct answer is $b$ then your answer is considered correct if
$$ \left| \frac{a-b}{a} \right| < 0.001$$

In [6]:
import numpy as np
def mapcos(A):
    return A.map(np.cos).collect()
mapcos(A)

[1.0, 0.54030230586813977, -0.41614683654714241]

In [7]:
def very_close_lists(A,B,tol=0.001):
    ''' Check that the two firs parameters are lists of equal length 
    and then check'''
    assert type(A)==list and type(B) ==list
    assert len(A)==len(B)
    for i in range(len(A)):
        a=A[i]; b=B[i]
        if a==0:
            a+=1; b+=1
        else:
            r=(a-b)/a
            assert abs(r)<tol
            
very_close_lists(mapcos(sc.parallelize(range(3))),[1.0, 0.5403, -0.4161])

### Excercise 2
Consider the following RDD: 

```
stringRDD=sc.parallelize(["Spring quarter", "Learning spark basics", "Big data analytics with Spark"])
```
   Write a function called `mapwords` that has a single paramater: an RDD of strings, and returns a list of words for each string.  
    ```mapwords(MapString)``` should produce the output:
    
``` 
[['Spring', 'quarter'], ['Learning', 'spark', 'basics'], ['Big', 'data', 'analytics', 'with', 'Spark']]
```

### Reduce

* Takes RDD as input, returns a single value.
* **Reduce operator** takes **two** elements as input returns **one** as output.
* Repeatedly applies a **reduce operator**
* Each executor reduces the data local to it.
* The results from all executors are combined.

The simplest example of a 2-to-1 operation is the sum:

In [8]:
A.reduce(lambda x,y: x+y)

3

Here is an example of a reduce operation that finds the shortest string in an RDD of strings.

In [9]:
words=['this','is','the','best','mac','ever']
wordRDD=sc.parallelize(words)
wordRDD.reduce(lambda w,v: w if len(w)<len(v) else v)

'is'

#### Exercise 2

1. Write a `reduce` command that outputs the maximum number from a list of numbers. Your command should produce the following output on the input ```RDD=sc.parallelize([0,2,1])```

   Output: ``` 2 ```
   

2. Consider the stringRDD defined in Exercise. Write a `reduce` command to produce a single string which is the concatenation of all the strings in stringRDD(with a space between each string). You output should look like:

    Output: ``` 'Spring quarter Learning spark basics Big data analytics with Spark' ```

### Using regular functions instead of lambda functions

* lambda function are short and sweet.
* but sometimes it's hard to use just one line.
* We can use full-fledged functions instead.

suppose we want to find the 
* last word in a lexicographical order 
* among 
* the longest words in the list.

We could achieve that as follows

In [10]:
def largerThan(x,y):
    if len(x)>len(y): return x
    elif len(y)>len(x): return y
    else:  #lengths are equal, compare lexicographically
        if x>y: 
            return x
        else: 
            return y
        
wordRDD.reduce(largerThan)

'this'

#### Exercise 3

1. Consider the following RDD:

    ``` listRDD=sc.parallelize([[3,4],[2,1],[7,9]]) ```
 
     Write a regular function with `reduce` command to output the maximum element from a set of lists. Your output should look like:
     
     Output: ```[9]```
     
     (Note: The output is a list containing a single number rather than just a single number)

<h3>Reduce operations <strong>must not depend on the order</strong></h3>

<ul>
	<li>Order of operands should not matter</li>
	<li>Order of application of reduce operator should not matter</li>
</ul>

<p>Multiplication and summation are good:</p>

<h1>&nbsp; &nbsp; &nbsp; &nbsp; 1 + 3 + 5 + 2 &nbsp; &nbsp; &nbsp;5 + 3 + 1 + 2 &nbsp;</h1>


<p>Division and subtraction are bad:</p>

<h1>&nbsp; &nbsp; &nbsp; &nbsp; 1 - 3 - 5 - 2 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;1 - 3 - 5 - 2 &nbsp;</h1>


### Why must reordering not change the result?

You can think about the reduce operation as a binary tree where the leaves are the elements of the list and the root is the final result. Each triplet of the form (parent, child1, child2) corresponds to a single application of the reduce function. 

The order in which the reduce operation is applied is **determined at run time** and depends on how the RDD is partitioned across the cluster.
There are many different orders to apply the reduce operation. 

If we want the input RDD to uniquely determine the reduced value **all evaluation orders must must yield the same final result**. In addition, the order of the elements in the list must not change the result. In particular, reversing the order of the operands in a reduce function must not change the outcome. 

For example the arithmetic operations multiply `*` and add `+` can be used in a reduce, but the operations subtract `-` and divide `/` should not.

Doing so will not raise an error, but the result is unpredictable.

Here is a example  
Which of these the following orders was executed?
* $$((1-3)-5)-2$$ or
* $$(1-3)-(5-2)$$

In [11]:
B=sc.parallelize([1,3,5,2])
B.reduce(lambda x,y: x-y)

-5